# Building LangChain Agents with Multiple LLM Providers along with Tracing from Langsmith

This notebook demonstrates how to create and use LangChain agents with different LLM providers (OpenAI and Together AI). You'll learn how agents use tools to answer queries by selecting the appropriate resources based on the context.

In [ ]:
# Install necessary dependencies
!pip install langchain langchain-openai langchain-together langchain-experimental langchain-community langsmith
!pip install wikipedia duckduckgo-search arxiv yfinance python-dotenv

# Note: You may need to restart the kernel after installation

# Setup and Dependencies
Import all necessary libraries including LangChain components, OpenAI integration, and Together AI integration. Explain what each import is used for in the context of building an agent.

In [ ]:
# Import necessary libraries
import os  # For setting environment variables

# Imnport LangSmith components
from langsmith import traceable # For tracing function calls

# Import LangChain components
from langchain.agents import initialize_agent, Tool  # For initializing the agent and defining tools
from langchain_openai import ChatOpenAI  # For OpenAI integration
from langchain_together import ChatTogether  # For Together AI integration
from langchain_experimental.utilities import PythonREPL  # For executing Python code
from langchain_community.utilities import WikipediaAPIWrapper  # For Wikipedia API wrapper
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool  # For Yahoo Finance News tool
from langchain_community.tools import (
    WikipediaQueryRun,  # For running Wikipedia queries
    DuckDuckGoSearchResults,  # For running DuckDuckGo search queries
    ArxivQueryRun,  # For running arXiv queries
)

# Set the USER_AGENT environment variable to avoid warnings
os.environ["USER_AGENT"] = "Chapter_10_Agent/1.0"

# Set the Langsmith env variables

os.environ["LANGSMITH_TRACE"] = "True"  # Enable tracing
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"  # Set the LangSmith endpoint
os.environ["LANGSMITH_API_KEY"] = ""  # Enter Your LangSmith API Key
os.environ["LANGSMITH_PROJECT"] = "chapter-10"  # Set the LangSmith project


USER_AGENT environment variable not set, consider setting it to identify your requests.


# Configure API Keys
Set up environment variables for both OpenAI and Together AI API keys. Include code to check if keys are properly configured and instructions on how to obtain these keys.

In [ ]:
# Configure API Keys

# Set up environment variables for OpenAI and Together AI API keys
os.environ["OPENAI_API_KEY"] = ""
os.environ["TOGETHER_API_KEY"] = ""

# Check if the API keys are properly configured
assert "OPENAI_API_KEY" in os.environ, "OpenAI API key is not set. Please set the OPENAI_API_KEY environment variable."
assert "TOGETHER_API_KEY" in os.environ, "Together AI API key is not set. Please set the TOGETHER_API_KEY environment variable."

# Instructions on how to obtain these keys
# - OpenAI API Key: Sign up at https://beta.openai.com/signup/ and generate an API key from the API section.
# - Together AI API Key: Sign up at https://together.ai/ and generate an API key from the API section.

# Define Available Tools
Create and configure the tools that will be available to the agent: Wikipedia, DuckDuckGo search, ArXiv query, Python REPL, and Yahoo Finance News. Include explanations of what each tool does and when it should be used.

In [4]:
# Instantiate the Wikipedia API wrapper and create the tool
wiki_api = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

# Define the tools with clear descriptions
tools = [
    wiki_tool,
    DuckDuckGoSearchResults(),
    ArxivQueryRun(),
    Tool(
        name="Python REPL",
        func=PythonREPL().run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    ),
    YahooFinanceNewsTool(),
]

# Explanation:
# - WikipediaQueryRun: Uses the Wikipedia API to fetch information from Wikipedia.
# - DuckDuckGoSearchResults: Uses DuckDuckGo to fetch search results.
# - ArxivQueryRun: Uses arXiv to fetch scientific research papers.
# - Python REPL: Executes Python code snippets.
# - YahooFinanceNewsTool: Fetches financial news from Yahoo Finance.

# Create Model Provider Function
Implement a function that creates an LLM based on the provider choice (OpenAI or Together AI). Include parameters for model selection, temperature, and other relevant settings for each provider.

In [5]:
# Create Model Provider Function
def create_llm(provider: str, model: str, temperature: float = 0, max_tokens: int = None, timeout: int = None, max_retries: int = 2):
    """
    Create an LLM based on the provider choice (OpenAI or Together AI).
    
    Parameters:
    - provider (str): The LLM provider ('openai' or 'together').
    - model (str): The model name to use.
    - temperature (float): The temperature setting for the model.
    - max_tokens (int): The maximum number of tokens for the response.
    - timeout (int): The timeout setting for the model.
    - max_retries (int): The maximum number of retries for the model.
    
    Returns:
    - llm: The initialized LLM.
    """
    if provider == "openai":
        return ChatOpenAI(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            timeout=timeout,
            max_retries=max_retries,
        )
    elif provider == "together":
        return ChatTogether(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            timeout=timeout,
            max_retries=max_retries,
        )
    else:
        raise ValueError("Unsupported provider. Choose either 'openai' or 'together'.")

# Example usage:
# llm_openai = create_llm(provider="openai", model="gpt-4o-mini")
# llm_together = create_llm(provider="together", model="together-gpt-neoxt-chat")

# Define System Message
Create the system message that instructs the agent on how to select the appropriate tool based on the query type. Explain how the system message influences agent behavior.

In [6]:
# Define the system message that instructs the agent on how to select the appropriate tool based on the query type
system_message = (
    "You are Charlie, a helpful intelligent agent that selects the most appropriate tool for each query and provides the best response. "
    "If the query asks for financial analysis (especially stock-related news), use the Yahoo Finance News tool. "
    "For general information, use Wikipedia or DuckDuckGo. "
    "If the query is about scientific papers or research (for example, papers on LLMs), use arXiv. "
    "For executing code or calculations, use the PythonREPL tool. "
    "Always decide based on the context of the user's request."
)

# Explanation:
# - The system message guides the agent on which tool to use based on the type of query.
# - It ensures that the agent selects the most appropriate tool for financial analysis, general information, scientific research, or code execution.
# - This message influences the agent's behavior by providing clear instructions on tool selection.

# Agent Initialization
Create a function that initializes the agent with the chosen LLM provider and tools. Include explanation of how the zero-shot-react-description agent works and its decision-making process.

In [7]:
# Create Agent Initialization Function
def initialize_agent_with_provider(provider: str, model: str):
    """
    Initialize the agent with the chosen LLM provider and tools.
    
    Parameters:
    - provider (str): The LLM provider ('openai' or 'together').
    - model (str): The model name to use.
    
    Returns:
    - agent: The initialized agent.
    """
    # Create the LLM based on the provider
    llm = create_llm(provider=provider, model=model)
    
    # Initialize the agent with the LLM and tools
    agent = initialize_agent(
        tools,
        llm,
        agent="zero-shot-react-description",
        verbose=True,
        agent_kwargs={"system_message": system_message},
    )
    
    return agent

# Explanation:
# - This function initializes the agent with the chosen LLM provider (OpenAI or Together AI) and the predefined tools.
# - It uses the `create_llm` function to create the LLM based on the provider and model.
# - The `initialize_agent` function sets up the agent with the LLM, tools, and system message.
# - The agent is configured to use the "zero-shot-react-description" agent type, which selects the appropriate tool based on the query.
# - The `verbose` parameter is set to True to provide detailed output during the agent's operation.

# Example usage:
# agent_openai = initialize_agent_with_provider(provider="openai", model="gpt-4o-mini")
# agent_together = initialize_agent_with_provider(provider="together", model="together-gpt-neoxt-chat")

# Invoke function with Trace

Convert the agent invoke method into a function to apply tracing

In [ ]:
# Wrap the agent call with @traceable so the entire chain is logged.
@traceable
def run_agent(agent, query: str):
    return agent.invoke(query)

# Financial News Query Test
Run a financial news query with both providers and display results. Include visualization of the agent's thought process and explanation of why it selected particular tools.

In [ ]:
# Financial News Query Test

# Initialize agents for both OpenAI and Together AI
agent_openai = initialize_agent_with_provider(provider="openai", model="gpt-4o-mini")
agent_together = initialize_agent_with_provider(provider="together", model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

## Add your custom queries here:

# Run a financial news query with OpenAI 
print("=== Financial News Query with OpenAI ===")
result_openai = run_agent(agent_openai , "What happened today with Microsoft stocks?")
print("Final Answer (OpenAI):", result_openai)

# Run a financial news query with Together AI
print("\n=== Financial News Query with Together AI ===")
result_together = run_agent(agent_together, "What happened today with Microsoft stocks?")
print("Final Answer (Together AI):", result_together)

# Explanation:
# - This section initializes agents for both OpenAI and Together AI using the `initialize_agent_with_provider` function.
# - It runs a financial news query ("What happened today with Microsoft stocks?") with both agents.
# - The results from both agents are printed for comparison.

# Scientific Research Query Test
Run a scientific research query with both providers and display results. Analyze how the agent determines which tool to use for research queries.

In [ ]:
# Scientific Research Query Test

# Initialize agents for both OpenAI and Together AI
agent_openai = initialize_agent_with_provider(provider="openai", model="gpt-4o-mini")
agent_together = initialize_agent_with_provider(provider="together", model="together-gpt-neoxt-chat")

# Run a scientific research query with OpenAI
print("\n=== Scientific Research Query with OpenAI ===")
result_openai = run_agent(agent_openai, "Find recent research papers on large language models.")
print("Final Answer (OpenAI):", result_openai)

# Run a scientific research query with Together AI
print("\n=== Scientific Research Query with Together AI ===")
result_together = run_agent(agent_together, "Find recent research papers on large language models.")
print("Final Answer (Together AI):", result_together)

# Explanation:
# - This section initializes agents for both OpenAI and Together AI using the `initialize_agent_with_provider` function.
# - It runs a scientific research query ("Find recent research papers on large language models.") with both agents.
# - The results from both agents are printed for comparison.
# - This demonstrates how the agent determines which tool to use for research queries based on the system message instructions.

# Code Execution Query Test
Run a code execution query with both providers and display results. Explain how the agent recognizes computational tasks and safely executes Python code.

In [ ]:
# Code Execution Query Test

# Initialize agents for both OpenAI and Together AI
agent_openai = initialize_agent_with_provider(provider="openai", model="gpt-4o-mini")
agent_together = initialize_agent_with_provider(provider="together", model="together-gpt-neoxt-chat")

# Run a code execution query with OpenAI
print("\n=== Code Execution Query with OpenAI ===")
result_openai = run_agent(agent_openai, "Run a quick Python snippet to compute 2+2.")
print("Final Answer (OpenAI):", result_openai)

# Run a code execution query with Together AI
print("\n=== Code Execution Query with Together AI ===")
result_together = run_agent(agent_together, "Run a quick Python snippet to compute 2+2.")
print("Final Answer (Together AI):", result_together)

# Explanation:
# - This section initializes agents for both OpenAI and Together AI using the `initialize_agent_with_provider` function.
# - It runs a code execution query ("Run a quick Python snippet to compute 2+2.") with both agents.
# - The results from both agents are printed for comparison.
# - This demonstrates how the agent recognizes computational tasks and safely executes Python code using the Python REPL tool.

# Provider Comparison
Compare the results from OpenAI and Together AI for the same queries. Analyze differences in reasoning patterns, tool selection behavior, and response quality between providers.

In [ ]:
# Provider Comparison

# Initialize agents for both OpenAI and Together AI
agent_openai = initialize_agent_with_provider(provider="openai", model="gpt-4o-mini")
agent_together = initialize_agent_with_provider(provider="together", model="together-gpt-neoxt-chat")

# Define queries to test
queries = [
    "What happened today with Microsoft stocks?",
    "Find recent research papers on large language models.",
    "Run a quick Python snippet to compute 2+2."
]

# Function to run queries and compare results
def compare_providers(queries):
    for query in queries:
        print(f"\n=== Query: {query} ===")
        
        # Run query with OpenAI
        print("\n--- OpenAI ---")
        result_openai = run_agent(agent_openai, query)
        print("Final Answer (OpenAI):", result_openai)
        
        # Run query with Together AI
        print("\n--- Together AI ---")
        result_together = run_agent(agent_together, query)
        print("Final Answer (Together AI):", result_together)

# Run the comparison
compare_providers(queries)

# Explanation:
# - This section initializes agents for both OpenAI and Together AI using the `initialize_agent_with_provider` function.
# - It defines a list of queries to test the agents' responses.
# - The `compare_providers` function runs each query with both agents and prints the results for comparison.
# - This allows users to analyze differences in reasoning patterns, tool selection behavior, and response quality between OpenAI and Together AI.